In [1]:
import numpy as np
import matplotlib
import matplotlib.pyplot as plt
from matplotlib.transforms import Affine2D
from matplotlib.projections import PolarAxes
import mpl_toolkits.axisartist.floating_axes as floating_axes
import mpl_toolkits.axisartist.angle_helper as angle_helper
from mpl_toolkits.axisartist.grid_finder import FixedLocator, MaxNLocator, DictFormatter

from astropy.table import Table
from astropy.io import fits
from astropy.cosmology import FlatLambdaCDM, z_at_value

from scipy.spatial import cKDTree

In [2]:
#galaxy data file
gdata = fits.open("/home/asgrk/Downloads/vsquared/ALL.fits")
#VoidFinder maximal sphere output
vfdata = Table.read("/home/asgrk/Downloads/DR7_comoving_maximal.txt",format='ascii.commented_header')
#VoidFinder hole output
vfdata2 = Table.read("/home/asgrk/Downloads/DR7_comoving_holes.txt",format='ascii.commented_header')
#Vsquared triangle output
tridata = Table.read("/home/asgrk/Downloads/output_nov1/DR7_triangles.dat",format='ascii.commented_header')

In [3]:
D2R = np.pi/180.

def toSky(cs):
    c1  = cs.T[0]
    c2  = cs.T[1]
    c3  = cs.T[2]
    r   = np.sqrt(c1**2.+c2**2.+c3**2.)
    dec = np.arcsin(c3/r)/D2R
    ra  = (np.arccos(c1/np.sqrt(c1**2.+c2**2.))*np.sign(c2)/D2R)%360
    return r,ra,dec

In [4]:
gr   = gdata[1].data['Rgal']
gra  = gdata[1].data['ra']
gdec = gdata[1].data['dec']

vfx = vfdata['x']
vfy = vfdata['y']
vfz = vfdata['z']
vfr,vfra,vfdec = toSky(np.array([vfx,vfy,vfz]).T)
vfrad = vfdata['radius']
vfc  = matplotlib.cm.nipy_spectral(np.linspace(0,1,len(vfr)))
vfcc = np.random.choice(range(len(vfc)),len(vfc),replace=False)

vflag = vfdata2['flag']
vfx2 = vfdata2['x']
vfy2 = vfdata2['y']
vfz2 = vfdata2['z']
vfr1,vfra1,vfdec1 = toSky(np.array([vfx2,vfy2,vfz2]).T)
vfrad1 = vfdata2['radius']
vfr2   = [vfr1[vflag==vfl] for vfl in np.unique(vflag)]
vfra2  = [vfra1[vflag==vfl] for vfl in np.unique(vflag)]
vfdec2 = [vfdec1[vflag==vfl] for vfl in np.unique(vflag)]
vfrad2 = [vfrad1[vflag==vfl] for vfl in np.unique(vflag)]

p1_r,p1_ra,p1_dec = toSky(np.array([tridata['p1_x'],tridata['p1_y'],tridata['p1_z']]).T)
p2_r,p2_ra,p2_dec = toSky(np.array([tridata['p2_x'],tridata['p2_y'],tridata['p2_z']]).T)
p3_r,p3_ra,p3_dec = toSky(np.array([tridata['p3_x'],tridata['p3_y'],tridata['p3_z']]).T)
trivids = np.array(tridata['void_id'])
v2c  = matplotlib.cm.nipy_spectral(np.linspace(0,1,np.amax(trivids)+1))
v2cc = np.random.choice(range(len(v2c)),len(v2c),replace=False)

In [5]:
#calculate radii of maximal sphere-slice intersections
def cint(dec):
    cr = []
    for i in range(len(vfr)):
        dtd = np.abs(vfr[i]*np.sin((vfdec[i]-dec)*D2R))
        if dtd>vfrad[i]:
            cr.append(0.)
        else:
            cr.append(np.sqrt(vfrad[i]**2.-dtd**2.))
    return cr

#calculate radii of hole-slice intersections
def cint2(dec):
    cr = []
    for i in range(len(vfr2)):
        cr.append([])
        for j in range(len(vfr2[i])):
            dtd = np.abs(vfr2[i][j]*np.sin((vfdec2[i][j]-dec)*D2R))
            if dtd>vfrad2[i][j]:
                cr[i].append(0.)
            else:
                cr[i].append(np.sqrt(vfrad2[i][j]**2.-dtd**2.))
    return cr

#calculate coordinates of triangle-slice intersections
def trint(dec):
    decsum = np.array([(p1_dec>dec).astype(int),(p2_dec>dec).astype(int),(p3_dec>dec).astype(int)]).T
    intr  = [[] for _ in range(np.amax(trivids)+1)]
    intra = [[] for _ in range(np.amax(trivids)+1)]
    for i in range(len(trivids)):
        if np.sum(decsum[i])==0:
            continue
        if np.sum(decsum[i])==3:
            continue
        cv = trivids[i]
        if np.sum(decsum[i])==1:
            if decsum[i][0]==1:
                intr[cv].append((p1_r[i]+p2_r[i])/2.)
                intr[cv].append((p1_r[i]+p3_r[i])/2.)
                intra[cv].append((p1_ra[i]+p2_ra[i])/2.)
                intra[cv].append((p1_ra[i]+p3_ra[i])/2.)
            elif decsum[i][1]==1:
                intr[cv].append((p2_r[i]+p1_r[i])/2.)
                intr[cv].append((p2_r[i]+p3_r[i])/2.)
                intra[cv].append((p2_ra[i]+p1_ra[i])/2.)
                intra[cv].append((p2_ra[i]+p3_ra[i])/2.)
            elif decsum[i][2]==1:
                intr[cv].append((p3_r[i]+p1_r[i])/2.)
                intr[cv].append((p3_r[i]+p2_r[i])/2.)
                intra[cv].append((p3_ra[i]+p1_ra[i])/2.)
                intra[cv].append((p3_ra[i]+p2_ra[i])/2.)
        elif np.sum(decsum[i])==2:
            if decsum[i][0]==0:
                intr[cv].append((p1_r[i]+p2_r[i])/2.)
                intr[cv].append((p1_r[i]+p3_r[i])/2.)
                intra[cv].append((p1_ra[i]+p2_ra[i])/2.)
                intra[cv].append((p1_ra[i]+p3_ra[i])/2.)
            elif decsum[i][1]==0:
                intr[cv].append((p2_r[i]+p1_r[i])/2.)
                intr[cv].append((p2_r[i]+p3_r[i])/2.)
                intra[cv].append((p2_ra[i]+p1_ra[i])/2.)
                intra[cv].append((p2_ra[i]+p3_ra[i])/2.)
            elif decsum[i][2]==0:
                intr[cv].append((p3_r[i]+p1_r[i])/2.)
                intr[cv].append((p3_r[i]+p2_r[i])/2.)
                intra[cv].append((p3_ra[i]+p1_ra[i])/2.)
                intra[cv].append((p3_ra[i]+p2_ra[i])/2.)
    return intr,intra

In [6]:
#convert a circle's coordinates to ordered boundary
def gcp(cc1,cc2,crad,npt):
    ccx = cc1*np.cos(cc2*D2R)
    ccy = cc1*np.sin(cc2*D2R)
    Cx = np.linspace(0.,2*np.pi,npt)
    Cy = np.linspace(0.,2*np.pi,npt)
    Cx = np.cos(Cx)*crad+ccx
    Cy = np.sin(Cy)*crad+ccy
    C1 = np.sqrt(Cx**2.+Cy**2.)
    C2 = (np.sign(Cy)*np.arccos(Cx/C1)+np.pi*(1.-np.sign(Cy)))/D2R
    return C1,C2

#convert circles' coordinates to ordered boundary
def gcp2(cc1,cc2,crad,npt,chkdpth):
    ccx = cc1*np.cos(cc2*D2R)
    ccy = cc1*np.sin(cc2*D2R)
    Cx = [np.linspace(0.,2*np.pi,int(npt*crad[k]/10)) for k in range(len(ccx))]
    Cy = [np.linspace(0.,2*np.pi,int(npt*crad[k]/10)) for k in range(len(ccx))]
    Cx = [np.cos(Cx[k])*crad[k]+ccx[k] for k in range(len(ccx))]
    Cy = [np.sin(Cy[k])*crad[k]+ccy[k] for k in range(len(ccx))]
    for i in range(len(ccx)):
        for j in range(len(ccx)):
            if i==j:
                continue
            cut = (Cx[j]-ccx[i])**2.+(Cy[j]-ccy[i])**2.>crad[i]**2.
            Cx[j] = Cx[j][cut]
            Cy[j] = Cy[j][cut]
    Cp = []
    for i in range(len(ccx)):
        Cp.extend(np.array([Cx[i],Cy[i]]).T.tolist())
    Cp = np.array(Cp)
    kdt = cKDTree(Cp)
    Cpi = [0]
    while len(Cpi)<len(Cp):
        if len(Cpi)==1:
            nid = kdt.query(Cp[Cpi[-1]],2)[1][1]
        else:
            nids = kdt.query(Cp[Cpi[-1]],chkdpth+1)[1][1:]
            for k in range(chkdpth):
                if nids[k] not in Cpi[(-1*(chkdpth+1)):-1]:
                    nid = nids[k]
                    break
            nids = kdt.query(Cp[Cpi[-1]],7)[1][1:]
        Cpi.append(nid)
    #Cpi.append(0)
    C1 = np.sqrt(Cp[Cpi].T[0]**2.+Cp[Cpi].T[1]**2.)
    C2 = (np.sign(Cp[Cpi].T[1])*np.arccos(Cp[Cpi].T[0]/C1)+np.pi*(1.-np.sign(Cp[Cpi].T[1])))/D2R
    return C1,C2
    

#convert triangle-slice intersections to ordered boundary
def convint(intr):
    crid = 0
    intr2 = []
    chkln = len(np.unique(intr))
    while len(intr2)<chkln+1:
        intr2.append(intr[crid])
        chkloc = np.where(intr==intr[crid])[0]
        chkloc = chkloc[chkloc != crid]
        if crid%2==0:
            chkloc2 = np.where(intr==intr[crid+1])[0]
            chkloc2 = chkloc2[chkloc2 != crid+1]
        else:
            chkloc2 = np.where(intr==intr[crid-1])[0]
            chkloc2 = chkloc2[chkloc2 != crid-1]
        try:
            crid = chkloc2[0]
        except:
            crid = 0
    return intr2

In [7]:
def setup_axes3(fig, rect):
    """
    Sometimes, things like axis_direction need to be adjusted.
    """

    # rotate a bit for better orientation
    tr_rotate = Affine2D().translate(-95, 0)

    # scale degree to radians
    tr_scale = Affine2D().scale(np.pi/180., 1.)

    tr = tr_rotate + tr_scale + PolarAxes.PolarTransform()

    grid_locator1 = angle_helper.LocatorDMS(4)
    tick_formatter1 = angle_helper.FormatterDMS()
        
    grid_locator2 = MaxNLocator(3)

    ra0, ra1 = 108, 263
    cz0, cz1 = 0., 306.
    grid_helper = floating_axes.GridHelperCurveLinear(tr,
                                        extremes=(ra0, ra1, cz0, cz1),
                                        grid_locator1=grid_locator1,
                                        grid_locator2=grid_locator2,
                                        tick_formatter1=tick_formatter1,
                                        tick_formatter2=None,
                                        )

    ax1 = floating_axes.FloatingSubplot(fig, rect, grid_helper=grid_helper)
    fig.add_subplot(ax1)

    # adjust axis
    ax1.axis["left"].set_axis_direction("bottom")
    ax1.axis["right"].set_axis_direction("top")

    ax1.axis["bottom"].set_visible(False)
    ax1.axis["top"].set_axis_direction("bottom")
    ax1.axis["top"].toggle(ticklabels=True, label=True)
    ax1.axis["top"].major_ticklabels.set_axis_direction("top")
    ax1.axis["top"].label.set_axis_direction("top")

    ax1.axis["left"].label.set_text(r"r [Mpc h$^{-1}$]")
    ax1.axis["top"].label.set_text(r"$\alpha$")


    # create a parasite axes whose transData in RA, cz
    aux_ax = ax1.get_aux_axes(tr)

    aux_ax.patch = ax1.patch # for aux_ax to have a clip path as in ax
    ax1.patch.zorder=0.9 # but this has a side effect that the patch is
                        # drawn twice, and possibly over some other
                        # artists. So, we decrease the zorder a bit to
                        # prevent this.

    return ax1, aux_ax

In [8]:
#plot VoidFinder maximal spheres
def pvf(dec,wdth,npc):
    fig = plt.figure(1, figsize=(12,6))
    ax3, aux_ax3 = setup_axes3(fig, 111)
    Cr = cint(dec)
    for i in range(len(vfr)):
        if Cr[i]>0:
            Cr2,Cra2 = gcp(vfr[i],vfra[i],Cr[i],npc)
            aux_ax3.plot(Cra2,Cr2,color=vfc[vfcc[i]])
            aux_ax3.fill(Cra2,Cr2,alpha=0.5,color=vfc[vfcc[i]])
    gdcut = (gr*np.sin((gdec-dec)*D2R))**2.<wdth**2.
    aux_ax3.scatter(gra[gdcut],gr[gdcut],color='k',s=0.2)
    fig.tight_layout()
    fig.set_size_inches((12,6))
    plt.show()

#plot VoidFinder voids
def pvf2(dec,wdth,npc,chkdpth):
    fig = plt.figure(1, figsize=(12,6))
    ax3, aux_ax3 = setup_axes3(fig, 111)
    Cr = cint2(dec)
    for i in range(len(vfr)):
        if np.sum(Cr[i])>0:
            Cr2,Cra2 = gcp2(vfr2[i],vfra2[i],Cr[i],npc,chkdpth)
            aux_ax3.plot(Cra2,Cr2,color=vfc[vfcc[i]])
            aux_ax3.fill(Cra2,Cr2,alpha=0.5,color=vfc[vfcc[i]])
    gdcut = (gr*np.sin((gdec-dec)*D2R))**2.<wdth**2.
    aux_ax3.scatter(gra[gdcut],gr[gdcut],color='k',s=0.2)
    fig.tight_layout()
    fig.set_size_inches((12,6))
    plt.show()

#plot Vsquared voids
def pzbv(dec,wdth):
    fig = plt.figure(1, figsize=(12,6))
    ax3, aux_ax3 = setup_axes3(fig, 111)
    Intr,Intra = trint(dec)
    for i in range(np.amax(trivids)+1):
        if len(Intr[i])>0:
            Intr2 = convint(Intr[i])
            Intra2 = convint(Intra[i])
            aux_ax3.plot(Intra2,Intr2,color=v2c[v2cc[i]])
            aux_ax3.fill(Intra2,Intr2,alpha=0.5,color=v2c[v2cc[i]])
    gdcut = (gr*np.sin((gdec-dec)*D2R))**2.<wdth**2.
    aux_ax3.scatter(gra[gdcut],gr[gdcut],color='k',s=0.2)
    fig.tight_layout()
    fig.set_size_inches((12,6))
    plt.show()